In [1]:
#Ref: Langchain in your pocket, Mehul Gupta
import sys
import os
sys.path.append('/home/vino/api_keys')
from api_key import OPENAI_API_KEY, OWM_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

import warnings
warnings.filterwarnings('ignore')

#### Example:1

In [2]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import YouTubeSearchTool
import os

In [3]:
youtube =  YouTubeSearchTool()
wiki  = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [4]:
tools = [
    
    Tool(
        name="youtube",
        func=youtube.run,
        description="Helps in getting youtube videos",

    ),

    Tool(
        name="wiki",
        func=wiki.run,
        description="Useful to search about a popular entity",
    )
]

In [5]:
prefix = "Answer the question asked. You have access to the following tools:"
suffix = "Begin!\
{chat_history}\
Question: {input}\
{agent_scratchpad}"

In [6]:
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input","chat_history","agent_scratchpad"], )

memory = ConversationBufferMemory(memory_key="chat_history")

In [7]:
llm_chain=LLMChain(llm=OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0),prompt=prompt)
agent=ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

In [8]:
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory )

agent_chain.run(input="Tell me something about Goddess Saraswathi")



> Entering new AgentExecutor chain...

Thought: I should use the wiki tool to search for information about Goddess Saraswathi.
Action: wiki
Action Input: Goddess Saraswathi
Observation: Page: Saraswati
Summary: Saraswati (Sanskrit: सरस्वती, IAST: Sarasvatī), also spelled as Sarasvati, is one of the principal goddesses in Hinduism, revered as the goddess of knowledge, education, learning, arts, speech, poetry, music, purification, language and culture. Together with the goddesses Lakshmi and Parvati, she forms the trinity, known as the Tridevi. Sarasvati is a pan-Indian deity, venerated not only in Hinduism but also in Jainism and Buddhism.
She is one of the prominent goddesses in the Vedic tradition who retains her significance in later Hinduism. In the Vedas, her characteristics and attributes are closely connected with the Sarasvati River, making her one of the earliest examples of a river goddess in Indian tradition. As a deity associated with a river, Sarasvati is revered for her

'Goddess Saraswathi is a prominent deity in Hinduism, revered as the goddess of knowledge, education, learning, arts, speech, poetry, music, purification, language, and culture. She is often associated with the Saraswati River and is also known as the goddess of speech and creativity. She is worshipped in various forms and has temples dedicated to her all over India.'

In [9]:
#available tools
import langchain.tools as tools
print(tools.__all__)

['AINAppOps', 'AINOwnerOps', 'AINRuleOps', 'AINTransfer', 'AINValueOps', 'AIPluginTool', 'APIOperation', 'ArxivQueryRun', 'AzureCogsFormRecognizerTool', 'AzureCogsImageAnalysisTool', 'AzureCogsSpeech2TextTool', 'AzureCogsText2SpeechTool', 'AzureCogsTextAnalyticsHealthTool', 'BaseGraphQLTool', 'BaseRequestsTool', 'BaseSQLDatabaseTool', 'BaseSparkSQLTool', 'BaseTool', 'BearlyInterpreterTool', 'BingSearchResults', 'BingSearchRun', 'BraveSearch', 'ClickTool', 'CopyFileTool', 'CurrentWebPageTool', 'DeleteFileTool', 'DuckDuckGoSearchResults', 'DuckDuckGoSearchRun', 'E2BDataAnalysisTool', 'EdenAiExplicitImageTool', 'EdenAiObjectDetectionTool', 'EdenAiParsingIDTool', 'EdenAiParsingInvoiceTool', 'EdenAiSpeechToTextTool', 'EdenAiTextModerationTool', 'EdenAiTextToSpeechTool', 'EdenaiTool', 'ElevenLabsText2SpeechTool', 'ExtractHyperlinksTool', 'ExtractTextTool', 'FileSearchTool', 'GetElementsTool', 'GmailCreateDraft', 'GmailGetMessage', 'GmailGetThread', 'GmailSearch', 'GmailSendMessage', 'GoogleC

In [10]:
#availabe toolkits
import langchain.agents.agent_toolkits
print(langchain.agents.agent_toolkits.__all__)

['AINetworkToolkit', 'AmadeusToolkit', 'AzureCognitiveServicesToolkit', 'FileManagementToolkit', 'GmailToolkit', 'JiraToolkit', 'JsonToolkit', 'MultionToolkit', 'NasaToolkit', 'NLAToolkit', 'O365Toolkit', 'OpenAPIToolkit', 'PlayWrightBrowserToolkit', 'PowerBIToolkit', 'SlackToolkit', 'SteamToolkit', 'SQLDatabaseToolkit', 'SparkSQLToolkit', 'VectorStoreInfo', 'VectorStoreRouterToolkit', 'VectorStoreToolkit', 'ZapierToolkit', 'create_json_agent', 'create_openapi_agent', 'create_pbi_agent', 'create_pbi_chat_agent', 'create_spark_sql_agent', 'create_sql_agent', 'create_vectorstore_agent', 'create_vectorstore_router_agent', 'create_conversational_retrieval_agent', 'create_retriever_tool']


In [11]:
#type of agents in langchain

#### Example:2

In [12]:
### !pip install pyowm langchain-experimental==0.0.46 langchain==0.0.349
from pyowm.owm import OWM
from pyowm.utils.config import get_default_config
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents.tools import Tool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

In [21]:
def get_weather(city):

    config_dict = get_default_config()
    config_dict['language'] = 'en' # your language here, eg. French 
    owm = OWM(OWM_API_KEY, config_dict)
    mgr = owm.weather_manager()
    observation = mgr.weather_at_place('Hyderabad, India')
    temperature=str(observation.weather.temperature("celsius")["temp"])+'°C'
    humidity = str(observation.weather.humidity)+'%'
    wind = str(observation.weather.wind())+' m/s'
    status = observation.weather.detailed_status

    return{'temp':temperature,'humid':humidity,'wind':wind,'status':status}

In [14]:
class GetWeatherInput(BaseModel):

    """Inputs for get_weather"""
    city:str=Field(description="City name with country separated by comma")

class GetWeatherTool(BaseTool):

    name = "get_weather_details_of_a_city"
    
    description = """
        Useful to get weather details of a city.
        Mandatory input format is 'city, country'.
        """
    args_schema: Type[BaseModel] = GetWeatherInput

    def _run(self, city: str): 
        weather = get_weather(city)
        return weather

    def _arun(self, city: str): 
        raise NotImplementedError("this tool doesn't support async")
    

In [15]:
llm = ChatOpenAI()
tools = [GetWeatherTool()]

In [16]:
model = ChatOpenAI()
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent=PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [28]:
# own_api_key not working for me...
#agent.run('Should I visit Manali, India this December based on the weather conditions?')

In [33]:
### tool decorator
from langchain.tools import BaseTool, StructuredTool, tool 

@tool
def abc(query: str)->str: 
    """Tool description"""
    return

tools=[abc]